In [1]:
import pandas as pd
import requests
import re

Necessário preencher infos API_KEY e endereço_de_entrada e depois dar Run All e os resultados saem em dois arquivos excel


Simples!!

In [16]:
# Chave da API (colocar entre aspas)
API_KEY = "COLOCAR CHAVE DE API AQUI ENTRE ASPAS"

# Endereço que será usado como entrada para calcular a sua distancia e tempo para todos os endereços (colocar entre aspas)
endereço_de_entrada = 'Praça Roberto Gomes Pedrosa, 1 - Morumbi, São Paulo - SP, 05653-070' #ex: endereço exemplificado aqui é do estadio do Morumbi

In [6]:
def captar_distancia(origem, destino):
    # Monta a URL
    url = (
        f"https://maps.googleapis.com/maps/api/distancematrix/json"
        f"?origins={origem}&destinations={destino}"
        f"&mode=driving&language=pt-BR&key={API_KEY}"
    )

    # Faz a requisição
    response = requests.get(url)
    data = response.json()

    # Extrai distância e tempo
    if data["status"] == "OK":
        elemento = data["rows"][0]["elements"][0]
        if elemento["status"] == "OK":
            distancia_texto = elemento["distance"]["text"]   # ex: '3,4 km'
            distancia_valor = elemento["distance"]["value"] / 1000  # em km
            duracao_texto = elemento["duration"]["text"]     # ex: '12 mins'

            print(f"Distância: {distancia_texto} ({distancia_valor:.2f} km)")
            print(f"Duração: {duracao_texto}")
        else:
            print("Erro no cálculo da distância:", elemento["status"])
    else:
        print("Erro na API:", data["status"])

    return distancia_texto, duracao_texto

In [7]:
endereços = {
    '1. Chimichurri Parrilla (@chimi_parrilla, R$ 42 - R$ 48)':'Av. Prof. Alfonso Bovero, 730 - Perdizes, São Paulo - SP, 05019-010',
    '2. Elevado Conselheiro (@elevado_conselheiro, R$ 86)':'R. Conselheiro Ramalho, 800 - Bela Vista, São Paulo - SP, 01325-000',
    '3. Z Deli Restaurante (@zdelisandwiches, R$ 67)':'Alameda Lorena, 1689 - Jardim Paulista, São Paulo - SP, 01424-007',
    '4. Buzina (@buzinaburgers, R$ 33 - R$ 38)':'R. Padre Carvalho, 30 - Pinheiros, São Paulo - SP, 05429-040',
    '5. Frank & Charles (@frankcharles1885, R$ 41 - R$ 47)':'Rua Tinhorão, 130 - Higienópolis, São Paulo - SP, 01241-030',
    '6. The Pitchers (@thepitchers_burger_baseball, R$ 35 - R$ 38)':'R. Dr. Bacelar, 1155 - Vila Clementino, São Paulo - SP, 04026-002',
    '7. Holy Burger (@holyburgersp, R$ 59 - R$ 59)':'R. Dr. Cesário Mota Júnior, 527 - Vila Buarque, São Paulo - SP, 01221-020',
    '8. DCK Burger (@dckburger, R$ 36 - R$ 41) (1)':'R. Demétrio Ribeiro, 605 - Tatuapé, São Paulo - SP, 03332-000',
    '8. DCK Burger (@dckburger, R$ 36 - R$ 41) (2)':'Av. Zelina, 1110 - Vila Prudente, São Paulo - SP, 03143-002',
    '9. Z Deli Sandwiches (@zdelisandwiches, R$ 44 - R$ 48) (1)':'R. Francisco Leitão, 16 - Pinheiros, São Paulo - SP, 05414-020',
    '9. Z Deli Sandwiches (@zdelisandwiches, R$ 44 - R$ 48) (2)':'R. Leopoldo Couto Magalhães Júnior, 785 - Itaim Bibi, São Paulo - SP, 04542-011',
    '9. Z Deli Sandwiches (@zdelisandwiches, R$ 44 - R$ 48) (3)':'R. Oscar Freire, 164 - Jardim Paulista, São Paulo - SP, 01426-000',
    '9. Z Deli Sandwiches (@zdelisandwiches, R$ 44 - R$ 48) (4)':'R. Bento Freitas, 314 - Vila Buarque, São Paulo - SP, 01220-000',
    '10. Caledonia Whisky & Co. (@caledoniabar, R$ 60)':'R. Vupabussu, 309 - Pinheiros, São Paulo - SP, 05429-040',
    '11. La Borratxeria (R$ 40 - R$ 46) (1)':'R. João Moura, 541 - Pinheiros, São Paulo - SP, 05412-001',
    '11. La Borratxeria (R$ 40 - R$ 46) (2)':'R. Ramos Batista, 380 - Vila Olímpia, São Paulo - SP, 04552-020',
    '12. Ecully Charbon (R$ 42 - R$ 42)':'R. Min. Ferreira Alves, 825 - Pompeia, São Paulo - SP, 05009-060',
    '13. Xepa (R$ 50 - R$ 59)':'R. Jaguaribe, 529 - Consolação, São Paulo - SP, 01224-001',
    '14. Ici Bistrô (@ici_bistro, R$ 59)':'Rua Mato Grosso, 396, São Paulo, Brazil 01243-020',
    '15. Raw Burger (@raw.burger, R$ 47 - R$ 57)':'R. Aspicuelta, 176 - Vila Madalena, São Paulo - SP, 05433-010',
    '16. Casa Carbone (@casacarbonebr, R$ 38 - R$ 47) ':'R. Mourato Coelho, 1325 - Pinheiros, São Paulo - SP, 05417-012',
    '17. Parrillita (@parrillitabr, R$ 45 - R$ 45)':'Av. Lins de Vasconcelos, 988 - Cambuci, São Paulo - SP, 01538-000',
    '18. Cortés Asador (R$ 45 - R$ 54) (1)':'Av. das Nações Unidas, 4777 - Jardim Universidade Pinheiros, São Paulo - SP, 05477-000',
    '18. Cortés Asador (R$ 45 - R$ 54) (2)':'144 Shopping Eldorado - Av. Rebouças, 3970 - Térreo - Loja - Pinheiros, São Paulo - SP, 05402-918',
    '19. Osso (R$ 64)':'R. Bandeira Paulista, 520 - Itaim Bibi, São Paulo - SP, 04532-001',
    '20. Patties (R$ 20 - R$ 22) (1)':'R. Flórida, 1420 - Brooklin, São Paulo - SP, 04565-001',
    '20. Patties (R$ 20 - R$ 22) (2)':'R. dos Pinheiros, 476 - Pinheiros, São Paulo - SP, 05422-001',
    '21. Debetti (@debetti, R$ 30 - R$ 46) (1)':'Av.Brig. Luiz Antonio, 3935 - Jardins',
    '21. Debetti (@debetti, R$ 30 - R$ 46) (2)':'R. Curumins, 11 - Cidade Jardim, São Paulo - SP, 05672-010',
    '22. Incêndio (@incendiorestaurante, R$ 45)':'Rua dos Pinheiros, 808, Pinheiros, Sao Paulo, Brazil 05422001',
    '23. Pão com Carne (@paocomcarne_hamburgueria, R$ 33 - R$ 36)':'R. Bandeira Paulista, 478 - Itaim Bibi, São Paulo - SP, 04532-002',
    '24. The Dog Haus (R$ 32 - R$ 35)':'R. Bandeira Paulista, 400 - Itaim Bibi, São Paulo - SP, 04532-000',
    '25. Coa Bar y Parrilla (@coa.bar, R$ 49)':'Alameda dos Anapurus, 1631 - Indianópolis, São Paulo - SP, 04087-006',
    '26. The Bear Burger (R$ 41 - R$ 42)':'R. Caiubí, 1537 - Perdizes, São Paulo - SP, 05010-000',
    '27. Beef (@beef_burger_beer, R$ 43 - R$ 42)':'Rua Antônio Pereira de Souza, 398.',
    '28. Lanchonete da Cidade (R$ 39 - R$ 43) (1)':'Av. Macuco, 355 - Moema - SP',
    '28. Lanchonete da Cidade (R$ 39 - R$ 43) (2)':'R. Coropés, 51 - Pinheiros - SP',
    '28. Lanchonete da Cidade (R$ 39 - R$ 43) (3)':'Av. Magalhães de Castro, 12.000 Piso 1, Shopping Cidade Jardim - Morumbi - SP',
    '28. Lanchonete da Cidade (R$ 39 - R$ 43) (4)':'Av. Higienópolis, 618 Piso Veiga Filho - Higienópolis - SP',
    '29. Big Kahuna (@bigkahunaburgerr, R$ 30 - R$ 40)':'Alameda Lorena, 53 - Jardim Paulista, São Paulo - SP, 01424-001',
    '30. Pantchos (@pantchoshouse, R$ 22 - R$ 37) (1)':'R. Ramos Batista, 409 - Vila Olímpia',
    '30. Pantchos (@pantchoshouse, R$ 22 - R$ 37) (2)':'Av Manuel Alves Soares 404',
    '31. BLS (R$ 28 - R$ 43) (1)':'R. Min. Jesuíno Cardoso, 149 - Itaim Bibi, São Paulo - SP, 04544-050',
    '31. BLS (R$ 28 - R$ 43) (2)':'R. Dona Ana Neri, 391',
    '32. Donnies (R$ 20 - R$ 23)':'R. João Cachoeira, 856 - Vila Nova Conceição',
    '33. Salt Burguer N Bar (R$ 39 - R$ 43)':'R. Augusto Tolle, 175 - Santana, São Paulo - SP, 02405-000',
    '34. Seven Kings (R$ 25 - R$ 44) (1)':'Alameda dos Anapurus, 1184 - Indianópolis, São Paulo - SP, 04087-003',
    '34. Seven Kings (R$ 25 - R$ 44) (2)':'R. Dr. Cândido Espinheira, 812 - Perdizes, São Paulo - SP, 05004-000',
    '35. Ritz (R$67 - R$ 77) (1)':'Al. Franca, 1088 - Jardim Paulista',
    '35. Ritz (R$67 - R$ 77) (2)':'Av. Dr. Chucri Zaidan, 902 - Vila Cordeiro',
    '35. Ritz (R$67 - R$ 77) (3)':'Brig. Faria Lima, 2232 - Jd. Paulistano',
    '36. Matilda Lanches (@matildalanches, R$ 29 - R$ 41)':'R. Mateus Grou, 31 - Pinheiros, São Paulo - SP, 05415-050',
    '37. Na Garagem (@nagaragem.hamburgueria, R$ 27 - R$ 31)':'Rua Benjamim Egas, 301 Pinheiros, São Paulo, Brazil 05418-030',
    '38. Coringa do Beco (R$ 35 - R$ 36)':'Rua Harmonia, 68a - Sumarezinho, São Paulo - SP, 05435-000',
    '39. All In Burger (R$ 32 - R$ 42)':'R. Dr. Renato Paes de Barros, 465 - Itaim Bibi, São Paulo - SP, 04530-000',
    '40. Castro Burger (R$ 34 - R$ 36)':'R. Joaquim Távora, 1517 - Vila Mariana, São Paulo - SP, 04015-003',
    '41. Tradissa (R$ 33 - R$ 35)':'Av. Gen. Ataliba Leonel, 3477 - Parada Inglesa, São Paulo - SP, 02242-001',
    '42. Jota Hamburgers (@jotahamburgers, R$ 16 - R$ 18)':'R. Pamplona, 529 - Jardim Paulista, São Paulo - SP, 01405-100',
}

In [8]:
# DataFrame com uma linha por par (chave, valor)
df = pd.DataFrame(list(endereços.items()), columns=['Restaurante', 'Endereço Completo'])
df

,Restaurante,Endereço Completo
0,"1. Chimichurri Parrilla (@chimi_parrilla, R$ 4...","Av. Prof. Alfonso Bovero, 730 - Perdizes, São ..."
1,"2. Elevado Conselheiro (@elevado_conselheiro, ...","R. Conselheiro Ramalho, 800 - Bela Vista, São ..."
2,"3. Z Deli Restaurante (@zdelisandwiches, R$ 67)","Alameda Lorena, 1689 - Jardim Paulista, São Pa..."
3,"4. Buzina (@buzinaburgers, R$ 33 - R$ 38)","R. Padre Carvalho, 30 - Pinheiros, São Paulo -..."
4,"5. Frank & Charles (@frankcharles1885, R$ 41 -...","Rua Tinhorão, 130 - Higienópolis, São Paulo - ..."
5,"6. The Pitchers (@thepitchers_burger_baseball,...","R. Dr. Bacelar, 1155 - Vila Clementino, São Pa..."
6,"7. Holy Burger (@holyburgersp, R$ 59 - R$ 59)","R. Dr. Cesário Mota Júnior, 527 - Vila Buarque..."
7,"8. DCK Burger (@dckburger, R$ 36 - R$ 41) (1)","R. Demétrio Ribeiro, 605 - Tatuapé, São Paulo ..."
8,"8. DCK Burger (@dckburger, R$ 36 - R$ 41) (2)","Av. Zelina, 1110 - Vila Prudente, São Paulo - ..."
9,"9. Z Deli Sandwiches (@zdelisandwiches, R$ 44 ...","R. Francisco Leitão, 16 - Pinheiros, São Paulo..."


In [9]:
#Gerando KM e Tempo para todos os endereços
lista_distancias = [ ]
lista_duração = []
for i in range (len(df)):
    a, b = captar_distancia(endereço_de_entrada , df['Endereço Completo'][i])
    lista_distancias.append(a)
    lista_duração.append(b)
    

Distância: 11,9 km (11.92 km)
Duração: 29 minutos
Distância: 11,2 km (11.23 km)
Duração: 32 minutos
Distância: 9,0 km (9.04 km)
Duração: 23 minutos
Distância: 7,4 km (7.38 km)
Duração: 17 minutos
Distância: 10,8 km (10.83 km)
Duração: 27 minutos
Distância: 13,3 km (13.30 km)
Duração: 25 minutos
Distância: 11,5 km (11.50 km)
Duração: 31 minutos
Distância: 19,7 km (19.67 km)
Duração: 43 minutos
Distância: 22,6 km (22.57 km)
Duração: 43 minutos
Distância: 7,5 km (7.51 km)
Duração: 21 minutos
Distância: 6,7 km (6.73 km)
Duração: 19 minutos
Distância: 8,6 km (8.58 km)
Duração: 21 minutos
Distância: 11,3 km (11.29 km)
Duração: 31 minutos
Distância: 6,5 km (6.46 km)
Duração: 14 minutos
Distância: 9,1 km (9.11 km)
Duração: 22 minutos
Distância: 6,7 km (6.68 km)
Duração: 18 minutos
Distância: 11,7 km (11.75 km)
Duração: 28 minutos
Distância: 11,8 km (11.76 km)
Duração: 32 minutos
Distância: 11,1 km (11.06 km)
Duração: 29 minutos
Distância: 8,3 km (8.33 km)
Duração: 20 minutos
Distância: 8,1 km 

In [10]:
# Criando Colunas
df['Distância'] = lista_distancias
df['Tempo'] = lista_duração

# Remover " km" e trocar vírgula por ponto para virar número
df["Distancia (KM)"] = df["Distância"].str.replace(" km", "").str.replace(",", ".").astype(float)

In [11]:
def extrair_info(texto):
    # Regex corrigido
    padrao = r"(\d+)\.\s*([^(]+?)\s*\((?:@([^,]+),\s*)?R\$\s*(\d+)(?:\s*-\s*R\$\s*(\d+))?\)"
    match = re.search(padrao, texto)

    if match:
        rank = match.group(1)
        nome = match.group(2).strip()
        insta = f"@{match.group(3)}" if match.group(3) else None
        preco_min = match.group(4)
        preco_max = match.group(5) if match.group(5) else preco_min
        return pd.Series([rank, nome, insta, f"R$ {preco_min} - R$ {preco_max}"])
    else:
        return pd.Series([None, None, None, None])

# Aplicar no DataFrame
df[["Rank", "Nome Restaurante", "Instagram", "Preço"]] = df["Restaurante"].apply(extrair_info)

In [13]:
df_to_excel = df[['Nome Restaurante', 'Rank', 'Endereço Completo', 'Preço', 'Instagram']]
df_to_excel.to_excel('tabela_infos_restaurante.xlsx')

In [14]:
df_final = df[['Nome Restaurante', 'Rank', 'Endereço Completo', 'Distancia (KM)', 'Tempo', 'Preço', 'Instagram']]
df_final = df_final.sort_values(['Distancia (KM)'])

# Cria coluna "Existe esse restaurante mais perto"
df_final["Existe esse restaurante mais perto"] = df_final["Rank"].duplicated().map({False: "Não", True: "Sim"})

df_final

,Nome Restaurante,Rank,Endereço Completo,Distancia (KM),Tempo,Preço,Instagram,Existe esse restaurante mais perto
37,Lanchonete da Cidade,28,"Av. Magalhães de Castro, 12.000 Piso 1, Shoppi...",4.3,10 minutos,R$ 39 - R$ 43,None,Não
49,Ritz,35,"Av. Dr. Chucri Zaidan, 902 - Vila Cordeiro",4.4,11 minutos,R$ 67 - R$ 77,None,Não
28,Debetti,21,"R. Curumins, 11 - Cidade Jardim, São Paulo - S...",5.2,12 minutos,R$ 30 - R$ 46,@debetti,Não
31,The Dog Haus,24,"R. Bandeira Paulista, 400 - Itaim Bibi, São Pa...",6.0,14 minutos,R$ 32 - R$ 35,None,Não
25,Patties,20,"R. Flórida, 1420 - Brooklin, São Paulo - SP, 0...",6.0,15 minutos,R$ 20 - R$ 22,None,Não
42,BLS,31,"R. Min. Jesuíno Cardoso, 149 - Itaim Bibi, São...",6.3,16 minutos,R$ 28 - R$ 43,None,Não
44,Donnies,32,"R. João Cachoeira, 856 - Vila Nova Conceição",6.4,17 minutos,R$ 20 - R$ 23,None,Não
52,Na Garagem,37,"Rua Benjamim Egas, 301 Pinheiros, São Paulo, B...",6.5,18 minutos,R$ 27 - R$ 31,@nagaragem.hamburgueria,Não
13,Caledonia Whisky & Co.,10,"R. Vupabussu, 309 - Pinheiros, São Paulo - SP,...",6.5,14 minutos,R$ 60 - R$ 60,@caledoniabar,Não
26,Patties,20,"R. dos Pinheiros, 476 - Pinheiros, São Paulo -...",6.6,18 minutos,R$ 20 - R$ 22,None,Sim


In [15]:
df_final.to_excel('hamburguerSP_top42.xlsx')